In [1]:
!pip install -q transformers -U evaluate arabert #farasapy pyarabic emoji pystemmer optuna==2.3.0
! rm -rf BERT_hard
! mkdir -p BERT_hard

In [2]:
from arabert import preprocess 

In [4]:
import warnings, pandas as pd
warnings.simplefilter(action='ignore', category=FutureWarning)

data=pd.read_csv('HARD-reviews.tsv',sep='\t', header = 0 , encoding = 'utf-16')
#data = pd.read_csv("clean-HARD.csv", sep=',')
#data = data.drop('Unnamed: 0', axis=1)
#data = data.rename(columns={'sentences': 'review'})
data=data.drop(['nights','room type','user type','Hotel name','no'],axis=1)
data.head(5)


,rating,review
0,2,“ممتاز”. النظافة والطاقم متعاون.
1,5,استثنائي. سهولة إنهاء المعاملة في الاستقبال. ل...
2,5,استثنائي. انصح بأختيار الاسويت و بالاخص غرفه ر...
3,1,“استغرب تقييم الفندق كخمس نجوم”. لا شي. يستحق ...
4,4,جيد. المكان جميل وهاديء. كل شي جيد ونظيف بس كا...


In [5]:
import re
def clean(text):
    text =re.sub(r'[a-zA-Z?]', '', text).strip()

    return text

data['sentences'] = data['review'].apply(lambda x:clean(x))
data.head(3)

,rating,review,sentences
0,2,“ممتاز”. النظافة والطاقم متعاون.,“ممتاز”. النظافة والطاقم متعاون.
1,5,استثنائي. سهولة إنهاء المعاملة في الاستقبال. ل...,استثنائي. سهولة إنهاء المعاملة في الاستقبال. ل...
2,5,استثنائي. انصح بأختيار الاسويت و بالاخص غرفه ر...,استثنائي. انصح بأختيار الاسويت و بالاخص غرفه ر...


In [6]:
data['rating'].value_counts()

2    38467
4    26450
5    26399
1    14382
Name: rating, dtype: int64

In [7]:
data = data[['sentences','rating']]
data

,sentences,rating
0,“ممتاز”. النظافة والطاقم متعاون.,2
1,استثنائي. سهولة إنهاء المعاملة في الاستقبال. ل...,5
2,استثنائي. انصح بأختيار الاسويت و بالاخص غرفه ر...,5
3,“استغرب تقييم الفندق كخمس نجوم”. لا شي. يستحق ...,1
4,جيد. المكان جميل وهاديء. كل شي جيد ونظيف بس كا...,4
...,...,...
105693,“فند”. لا شئ عجبني. طقم العمل سيئ جدالا يوجد ب...,1
105694,“سيئ”. قربه من المسجد النبوي الشريف. استخدام م...,2
105695,“اسوأ إقامة في الرحلة !”. القرب من الحرم. كل ش...,2
105696,“دون المستوى”. قربه من الحرم. كل شيء,2


In [8]:
from arabert.preprocess import ArabertPreprocessor

model_name="bert-base-arabertv2"
arabert_prep = ArabertPreprocessor(model_name=model_name)


def arabertPreprocessor(text):
    text = arabert_prep.preprocess(text)
    return text

data['process_review'] = data['sentences'].apply(lambda x:arabertPreprocessor(x))
data

[2023-04-23 17:48:50,532 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.


,sentences,rating,process_review
0,“ممتاز”. النظافة والطاقم متعاون.,2,“ ممتاز ” . ال+ نظاف +ة و+ ال+ طاقم متعاون .
1,استثنائي. سهولة إنهاء المعاملة في الاستقبال. ل...,5,استثنائي . سهول +ة إنهاء ال+ معامل +ة في ال+ ا...
2,استثنائي. انصح بأختيار الاسويت و بالاخص غرفه ر...,5,استثنائي . انصح ب+ أختيار ال+ اسويت و ب+ ال+ ا...
3,“استغرب تقييم الفندق كخمس نجوم”. لا شي. يستحق ...,1,“ استغرب تقييم ال+ فندق ك+ خمس نجوم ” . لا شي ...
4,جيد. المكان جميل وهاديء. كل شي جيد ونظيف بس كا...,4,جيد . ال+ مكان جميل و+ هاديء . كل شي جيد و+ نظ...
...,...,...,...
105693,“فند”. لا شئ عجبني. طقم العمل سيئ جدالا يوجد ب...,1,“ فند ” . لا شئ عجبني . طقم ال+ عمل سيئ جدال +...
105694,“سيئ”. قربه من المسجد النبوي الشريف. استخدام م...,2,“ سيئ ” . قرب +ه من ال+ مسجد ال+ نبوي ال+ شريف...
105695,“اسوأ إقامة في الرحلة !”. القرب من الحرم. كل ش...,2,“ أسوأ إقام +ة في ال+ رحل +ة ! ” . ال+ قرب من ...
105696,“دون المستوى”. قربه من الحرم. كل شيء,2,“ دون ال+ مستوى ” . قرب +ه من ال+ حرم . كل شيء


In [9]:
data['process_review'].isnull().values.any(), data['process_review'].isnull().sum()

(False, 0)

In [10]:
data = data.dropna(subset = ['rating', 'process_review'])

In [11]:
data['process_review'].isnull().values.any(), data['process_review'].isnull().sum()

(False, 0)

In [12]:
data['label'] = pd.Categorical(data.rating, ordered=True).codes
data['label'].unique()

array([1, 3, 0, 2], dtype=int8)

In [13]:
mapLabels = pd.DataFrame(data.groupby(['rating', 'label']).count())

#drop count column
mapLabels.drop(['process_review'], axis = 1, inplace = True)
label2Index = mapLabels.to_dict(orient='index')

print (f"label2Index :{label2Index}")
print (type(label2Index))
#print (f"index2Label :{index2Label}")

label2Index :{(1, 0): {'sentences': 14382}, (2, 1): {'sentences': 38467}, (4, 2): {'sentences': 26450}, (5, 3): {'sentences': 26399}}
<class 'dict'>


In [14]:
index2label = {}

for key in label2Index:
    print (f"{key[1]} -> {key[0]}")
    index2label[key[1]] = key[0]

0 -> 1
1 -> 2
2 -> 4
3 -> 5


In [15]:
label2Index = {v: k for k, v in index2label.items()}

print (f'label2Index: {label2Index}')
print (f'index2label: {index2label}')

label2Index: {1: 0, 2: 1, 4: 2, 5: 3}
index2label: {0: 1, 1: 2, 2: 4, 3: 5}


In [16]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification

In [17]:
from transformers import BertTokenizer, BertForSequenceClassification ,BertModel
tokenizer = BertTokenizer.from_pretrained('aubmindlab/bert-base-arabertv2') #UBC-NLP/MARBERT #aubmindlab/bert-base-arabertv2
model = BertForSequenceClassification.from_pretrained('aubmindlab/bert-base-arabertv2', num_labels=4) #UBC-NLP/MARBERT

Some weights of the model checkpoint at aubmindlab/bert-base-arabertv2 were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were

In [18]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(64000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [19]:
X = list(data["process_review"])
y = list(data["label"])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1,stratify=y)
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)

In [20]:
# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels  

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [21]:
train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)

In [22]:
import evaluate
import numpy as np

def compute_metrics(eval_preds):
    metric = evaluate.load("accuracy")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [23]:
# Define Trainer
args = TrainingArguments(
    output_dir="BERT_hard",
    num_train_epochs=3,
    logging_steps=5000,
    save_steps=5000,
    save_total_limit=3,
    evaluation_strategy="epoch",
    report_to="none",
    learning_rate=2e-5,
    per_device_train_batch_size=16)


trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics)

In [24]:
history = trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.449300,0.391846,0.833396
2,0.381500,0.401877,0.830842
3,0.348900,0.424927,0.829612


In [25]:
eval_results = trainer.evaluate()

eval_ = pd.DataFrame().append(eval_results, ignore_index=True)
eval_ = eval_.rename(columns={"eval_loss": "Evaluation Loss", "eval_accuracy": "Evaluation Accuracy"})
eval_ = eval_[["Evaluation Loss", "Evaluation Accuracy"]] 
eval_.style.hide_index()

Evaluation Loss,Evaluation Accuracy
0.424927,0.829612


In [26]:
trainer.save_model("BERT_hard")